In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%ls

In [ ]:
!unzip '/content/drive/My Drive/Instagram/10.zip' -d '/content/drive/My Drive/Instagram/'

In [ ]:
import os
filelist =os.listdir('/content/drive/My Drive/Instagram/10/')
len(filelist)

In [ ]:
import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

import tensorflow as tf
import tensorflow_hub as hub

tf.get_logger().setLevel('ERROR')

## Visualization tools

To visualize the images with the proper detected boxes, keypoints and segmentation, we will use the TensorFlow Object Detection API. To install it we will clone the repo. Note: this was forked from 2020 The TensorFlow Hub Authors on Tensorflow Object Detection API. 


In [ ]:
#Note: this was forked from 2020 The TensorFlow Hub Authors
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  image = None
  if(path.startswith('http')):
    response = urlopen(path)
    image_data = response.read()
    image_data = BytesIO(image_data)
    image = Image.open(image_data)
  else:
    image_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(image_data))

  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (1, im_height, im_width, 3)).astype(np.uint8)




In [ ]:
# Clone the tensorflow models repository
!git clone --depth 1 https://github.com/tensorflow/models

Intalling the Object Detection API

In [ ]:
%%bash
sudo apt install -y protobuf-compiler
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install -q .

Now we can import the dependencies we will need later

In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

%matplotlib inline

### Load label map data (for plotting).

Label maps correspond index numbers to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine.

We are going, for simplicity, to load from the repository that we loaded the Object Detection API code

In [ ]:
PATH_TO_LABELS = './models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)


In [ ]:

model_display_name = 'SSD MobileNet v2 320x320'
model_handle = 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2'


In [ ]:
hub_model = hub.load(model_handle)
print('model loaded!')


In [ ]:
num_objects = np.zeros(8631,)

In [ ]:

import timeit


start = timeit.default_timer()


for i in range(0,8632):
  im = '/content/drive/My Drive/Instagram/10/' + filelist[i]
  image = load_image_into_numpy_array(im)
  t = hub_model(image)
  rr = {key:value.numpy() for key,value in t.items()}
  num_objects[i]=sum(rr['detection_scores'][0]>0.3)
  print(i)
  


stop = timeit.default_timer()

print('Time: ', stop - start)  
# different object detection models have additional results
# all of them are explained in the documentation


#Your statements here



In [ ]:
np.savetxt('/content/drive/My Drive/Instagram/10.csv', [p for p in zip(filelist, num_objects)], delimiter=',', fmt='%s')

In [ ]:

for result in results:    
# different object detection models have additional results
# all of them are explained in the documentation
  rr = {key:value.numpy() for key,value in result.items()}
  num_objects[i]=sum(rr['detection_scores'][0]>0.3)
  i+=1 
  print(i)

In [ ]:
for num in num_objects:
  print(num)

In [ ]:
num_objects[1234]